In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import datetime
import math

In [9]:
df = pd.read_csv('2018_03.csv')
df.dropna(subset=['delay_minutes'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
df_in = pd.read_csv('Inward.csv')
df_out = pd.read_csv('Outward.csv')

In [14]:
df.head()

,date,train_id,stop_sequence,from,from_id,to,to_id,scheduled_time,actual_time,delay_minutes,status,line,type
0,2018-03-01,3805,1.0,New York Penn Station,105,New York Penn Station,105,2018-03-02 01:22:00,2018-03-02 01:21:05,0.000000,departed,Northeast Corrdr,NJ Transit
1,2018-03-01,3805,2.0,New York Penn Station,105,Secaucus Upper Lvl,38187,2018-03-02 01:31:00,2018-03-02 01:31:08,0.133333,departed,Northeast Corrdr,NJ Transit
2,2018-03-01,3805,3.0,Secaucus Upper Lvl,38187,Newark Penn Station,107,2018-03-02 01:40:00,2018-03-02 01:40:07,0.116667,departed,Northeast Corrdr,NJ Transit
3,2018-03-01,3805,4.0,Newark Penn Station,107,Newark Airport,37953,2018-03-02 01:45:00,2018-03-02 01:45:10,0.166667,departed,Northeast Corrdr,NJ Transit
4,2018-03-01,3805,5.0,Newark Airport,37953,North Elizabeth,109,2018-03-02 01:49:00,2018-03-02 01:49:10,0.166667,departed,Northeast Corrdr,NJ Transit


In [15]:
lines_dict = {v:k for (k,v) in enumerate(df['line'].unique())}
lines_dict_inv = {k:v for (k,v) in enumerate(df['line'].unique())}
stations_dict = {v:k for (k,v) in enumerate(df['to'].unique())}
stations_dict_inv = {k:v for (k,v) in enumerate(df['to'].unique())}
stations_lines = {}
for i in range(len(df['line'].unique())):
    for j in range(len(df['from'][df['line']==lines_dict_inv[i]].unique())):
        stations_lines[df['from'][df['line']==lines_dict_inv[i]].unique()[j]] = df['line'].unique()[i]       

Only day number 1

In [16]:
idx = []
for (k,v) in enumerate(df['actual_time']):
    aux = datetime.datetime.strptime(v, '%Y-%m-%d %H:%M:%S')
    if aux.date() != datetime.date(2018, 3, 1):
        idx.append(k)
        
df1 = df.drop(idx)
df1.reset_index(drop=True, inplace=True)            

Split data in 24h

In [18]:
average_delays_hour = {}
hourly_delays_dataframe = {}
for i in range(0,24,1):
    average_delays_hour[i] = []
    for (k,v) in enumerate(df1['actual_time']):
        aux = datetime.datetime.strptime(v, '%Y-%m-%d %H:%M:%S')
        if aux.hour == i:
            average_delays_hour[i].append(k)
    hourly_delays_dataframe[i] = df1.iloc[average_delays_hour[i]]
    hourly_delays_dataframe[i].reset_index(drop=True, inplace=True)   
    
hourly_delays = {}
for j in range(0,24,1):
    hourly_delays[j] = []
    for i in range(len(stations_dict_inv)):
        hourly_delays[j].append(np.mean(hourly_delays_dataframe[j][hourly_delays_dataframe[j]['to'] == stations_dict_inv[i]]['delay_minutes']))
for k,v in hourly_delays.items():
    for i in range(len(v)):
        if math.isnan(v[i]):
            hourly_delays[k][i] = 0       

In [34]:
df_hour = pd.DataFrame(hourly_delays) 
df_hour

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0.0,0.050000,0.404545,0.045614,0.356790,2.166667,19.650000,...,6.052778,0.350000,0.211111,3.021333,5.433333,0.518519,0.111111,0.281944,1.900000,0.000000
1,0,0,0,0.0,0.500000,1.821212,1.814583,2.557018,3.344737,25.619048,...,6.665000,1.237879,2.268182,4.538542,4.951852,3.632143,1.436111,4.473077,3.730556,2.125000
2,0,0,0,0.0,1.316667,1.188889,1.458772,1.330128,1.103968,11.479630,...,1.951852,1.328205,1.034524,2.223333,3.811111,2.740476,1.488889,2.072917,1.893750,2.116667
3,0,0,0,0.0,1.570833,2.690476,5.507143,2.338095,2.423333,11.226667,...,2.866667,1.592593,1.740476,2.413333,3.329630,3.478333,1.395833,3.002778,2.150000,2.166667
4,0,0,0,0.0,1.750000,3.066667,1.737500,1.627778,1.416667,3.250000,...,1.200000,3.133333,0.708333,3.600000,4.016667,4.544444,1.888889,2.183333,2.811111,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0,0,0,0.0,0.000000,2.116667,1.166667,2.233333,2.211111,0.700000,...,1.116667,2.091667,2.216667,2.533333,3.033333,2.366667,2.225000,2.200000,4.250000,0.000000
161,0,0,0,0.0,0.000000,1.133333,0.233333,1.200000,1.183333,2.183333,...,0.583333,0.200000,2.183333,3.075000,3.050000,1.791667,3.250000,2.050000,4.116667,0.000000
162,0,0,0,0.0,0.000000,1.166667,0.000000,0.258333,1.205556,2.700000,...,1.533333,1.200000,2.700000,5.116667,5.366667,3.766667,5.216667,0.000000,3.100000,0.000000
163,0,0,0,0.0,0.000000,0.000000,1.233333,2.208333,2.758333,2.188889,...,2.616667,1.175000,2.150000,2.583333,3.411111,1.300000,2.300000,0.000000,2.175000,0.000000


Instead of 1 hour, split every 30 minutes. Half way through...dont take this code as true...still needs to be improved cause its not giving good results

In [41]:
period = 30 #minutes
average_delays_period = {}
period_delays_dataframe = {}
for i in range(0,24,1):
    for j in range(int(60/period)):
        average_delays_period[i+j] = []
        for (k,v) in enumerate(df1['actual_time']):
            aux = datetime.datetime.strptime(v, '%Y-%m-%d %H:%M:%S')
            if aux.hour == i and aux.minute >= period*j and aux.minute < period*(j+1):
                average_delays_period[i+j].append(k)
        period_delays_dataframe[i+j] = df1.iloc[average_delays_period[i+j]]
        period_delays_dataframe[i+j].reset_index(drop=True, inplace=True)   
    
period_delays = {}
for j in range(0,24,1):
    period_delays[j] = []
    for i in range(len(stations_dict_inv)):
        period_delays[j].append(np.mean(period_delays_dataframe[j][period_delays_dataframe[j]['to'] == stations_dict_inv[i]]['delay_minutes']))
for k,v in period_delays.items():
    for i in range(len(v)):
        if math.isnan(v[i]):
            period_delays[k][i] = 0  

In [42]:
len(period_delays_dataframe)

25